In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="deepseek-r1:1.5b")

response = llm.invoke("Explain quantum computing in simple words.")
print(response)

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Quantum computing is a revolutionary approach that harnesses principles from quantum mechanics to process information in ways that classical computers cannot. Here's a simple explanation:

1. **Qubits vs Bits**: Unlike regular bits, which are 0s and 1s (binary), qubits use something called "qubits." The key difference is that qubits can be in multiple states at once thanks to superposition.

2. **Superposition**: This means a quantum system isn't just one state; it's all possible states together. For an n-qubit system, there are 2^n possible states simultaneously, allowing for exponentially more possibilities to be processed.

3. **Entanglement**: When qubits are entangled, the state of one affects another, enabling strong correlations between them. This allows complex computations by connecting multiple qubits in a network.

4. **Computation Process**: Quantum computers use these principles to process an exponential number of calculations simultaneously. They can solve problems that c

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableParallel

In [12]:

model = Ollama(model="deepseek-r1:1.5b")

In [7]:

messages= [
    ("system", "You are expert about products, and you will help customer with product features."),
    ("human", "List 3 feature of product {product}")
]

In [19]:
prompt_template = ChatPromptTemplate.from_messages(messages)

In [13]:
def create_pros_list(product_details):
    messages= [
        ("system", "You are expert about products, and you will help customer with product pros."),
        ("human", "List 3 pros of product {product_details}")
    ]
    return ChatPromptTemplate.from_messages(messages)

In [14]:
def create_cons_list(product_details):
    messages= [
        ("system", "You are expert about products, and you will help customer with product cons."),
        ("human", "List 3 cons of product {product_details}")
    ]
    return ChatPromptTemplate.from_messages(messages)

In [15]:
def combine_pros_cons(pros, cons):
    return f"Pros : \n {pros} \n\n\n Cons : \n{cons}"

In [16]:
pros_parallel_chain = (
    RunnableSequence(lambda x : create_pros_list(x), model, StrOutputParser())
)

In [17]:
cons_parallel_chain = (
    RunnableSequence(lambda x : create_cons_list(x), model, StrOutputParser())
)

In [20]:
chain = (
    prompt_template
    |model
    |StrOutputParser()
    | RunnableParallel(branches ={"pros" : pros_parallel_chain, "cons" : cons_parallel_chain})
    | RunnableLambda(lambda x : combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"] ))
)

In [21]:
result = chain.invoke({"product" : " Latest Iphone"})
print("------------ Chain basic Example -------------")
print(result)

------------ Chain basic Example -------------
Pros : 
 The iPhone 15 Pro Max is a standout device that combines several high-quality features into one. Here's a concise breakdown:

- **High-Resolution OLED Display**: Enhances visuals with improved color accuracy, offering a polished and professional image quality.

- **Advanced Camera Technology**: Equipped with AI auto-tuning, this camera technology optimizes settings for face detection, background removal, or manual adjustments, reducing manual tweaking and saving time.

- **Comprehensive Connectivity**: Features Wi-Fi 6 (802.11ax) and Wi-Fi 5 (802.11n), along with Apple Watch integration, ensures fast streaming and seamless connectivity through Apple Music, making it ideal for devices like Apple Watches.

These features collectively offer a professional, efficient, and high-performing experience across various tasks, making the iPhone 15 Pro Max an excellent choice for users seeking both visual appeal and powerful tools. 


 Cons :